# Bias Uncovering Test Case (BTC) for Gender Bias

In [1]:
import os
import pandas as pd
import numpy as np
import math
import time
import pickle

from metric import evaluate_btc, evaluate_fairness_violation
from utils import load_pickle, calculate_test_accuracy, accuracy
from utils import load_original_prediction, load_mutant_and_prediction


## Measuring the BTC that can be found by BiasFinder

In [2]:
mutation_tool = "biasfinder"
bias_type = "gender"
task = "imdb"  # dataset used for fine-tuning
model = "bert-base-uncased"
mutant = "imdb"  # dataset used for generating mutants


df = load_mutant_and_prediction(mutation_tool, model, bias_type, mutant)

df.head()


,label,mutant,template,original,gender,template_id,prediction,original_prediction
0,0,"I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...",male,929,0,0
1,0,"I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...","I hated the book. A guy meets a smart dog, get...",female,929,0,0
2,1,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,male,1980,1,1
3,1,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,male,1980,1,1
4,1,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,The Great Dictator is a beyondexcellent film. ...,male,1980,1,1


In [3]:
print("Accuracy on mutants: {:.2f}%".format(accuracy(df["label"],df["prediction"])))


Accuracy on mutants: 93.29%


In [4]:
def print_evaluation(evaluation):
    print("# Mutants \t:", evaluation["mutant"])
    print("# Templates \t:", evaluation["template"])
    print("# BTCs \t\t:", evaluation["btc"])

evaluation = evaluate_btc(df["label"], df["prediction"], df["mutant"], df["template"], "gender", df["gender"])
print_evaluation(evaluation)

# Mutants 	: 153866
# Templates 	: 3015
# BTCs 		: 5543


## BTC on IMDB

In [5]:
# models = ["bert-base-uncased", "bert-base-cased", "roberta-base", "albert-base-v2", "microsoft/mpnet-base", "microsoft/deberta-base", "facebook/muppet-roberta-base", "google/electra-base-generator"]

models = ["bert-base-uncased"]

# models = ["bert-base-uncased", "bert-base-cased", "roberta-base", "xlnet-base-cased",
#           "albert-base-v2", "microsoft/mpnet-base", "microsoft/deberta-base",
#           "facebook/muppet-roberta-base", "google/electra-base-generator"]

mutation_tool = "biasfinder"
bias_type = "gender"
task = "imdb"  # dataset used for fine-tuning
mutant = "imdb"  # dataset used for generating mutants

d = pd.DataFrame(columns=["model", "tool",
                          "template", "mutant", "btc"])

for model in models :
    df = load_mutant_and_prediction(mutation_tool, model, bias_type, mutant)
    evaluation = evaluate_btc(df["label"], df["prediction"],
                            df["mutant"], df["template"], "gender", df["gender"])
    
    d = d.append(
            {
                "model": model, 
                "tool": mutation_tool, 
                "template": evaluation['template'], 
                "mutant": evaluation['mutant'],
                "btc" : evaluation['btc']
            }, 
            ignore_index=True)

d


,model,tool,template,mutant,btc
0,bert-base-uncased,biasfinder,3015,153866,5543


## BTC MTNLP

In [6]:
# mutation_tool = "mtnlp"
# bias_type = "gender"
# task = "imdb"  # dataset used for fine-tuning
# mutant = "imdb"  # dataset used for generating mutants

# df = load_mutant_and_prediction(mutation_tool, model, bias_type, mutant)
# evaluation = evaluate_btc(df["label"], df["prediction"],
#                           df["mutant"], df["template"], "gender", df["gender"])
# print_evaluation(evaluation)

In [7]:
models = ["bert-base-uncased"]

mutation_tools = ["biasfinder", "mtnlp"]
bias_type = "gender"
task = "imdb"  # dataset used for fine-tuning
mutant = "imdb"  # dataset used for generating mutants

d = pd.DataFrame(columns=["model", "tool",
                          "template", "mutant", "btc"])

for mutation_tool in mutation_tools:
    for model in models:
        df = load_mutant_and_prediction(
            mutation_tool, model, bias_type, mutant)
        evaluation = evaluate_btc(df["label"], df["prediction"],
                                  df["mutant"], df["template"], "gender", df["gender"])

        d = d.append(
            {
                "model": model,
                "tool": mutation_tool,
                "template": evaluation['template'],
                "mutant": evaluation['mutant'],
                "btc": evaluation['btc']
            },
            ignore_index=True)

d


,model,tool,template,mutant,btc
0,bert-base-uncased,biasfinder,3015,153866,5543
1,bert-base-uncased,mtnlp,95219,264602,305


## BTC on Twitter Semeval and Comparison with EEC

In [14]:
models = ["bert-base-uncased", "bert-base-cased", "roberta-base", "xlnet-base-cased",
          "albert-base-v2", "microsoft/mpnet-base", "microsoft/deberta-base",
          "facebook/muppet-roberta-base", "google/electra-base-generator"]

mutation_tools = ["biasfinder", "eec"]
bias_type = "gender"
task = "twitter_semeval"  # dataset used for fine-tuning
mutant = "twitter_semeval"  # dataset used for generating mutants

d = pd.DataFrame(columns=["model", "tool",
                          "template", "mutant", "btc"])

for mutation_tool in mutation_tools:
    for model in models:
        df = load_mutant_and_prediction(mutation_tool, model, bias_type, mutant)
        evaluation = evaluate_btc(df["label"], df["prediction"],
                                df["mutant"], df["template"], "gender", df["gender"])

        d = d.append(
            {
                "model": model,
                "tool": mutation_tool,
                "template": evaluation['template'],
                "mutant": evaluation['mutant'],
                "btc": evaluation['btc']
            },
            ignore_index=True)

d


FileNotFoundError: [Errno 2] No such file or directory: '../../data/biasfinder/gender/twitter_semeval/original.csv'

## BTC on Twitter S140 and Comparison with EEC

In [ ]:
models = ["bert-base-uncased", "bert-base-cased", "roberta-base", "xlnet-base-cased",
          "albert-base-v2", "microsoft/mpnet-base", "microsoft/deberta-base",
          "facebook/muppet-roberta-base", "google/electra-base-generator"]

mutation_tools = ["biasfinder", "eec"]
bias_type = "gender"
task = "twitter_s140"  # dataset used for fine-tuning
mutant = "twitter_s140"  # dataset used for generating mutants

d = pd.DataFrame(columns=["model", "tool",
                          "template", "mutant", "btc"])

for model in models:
    df = load_mutant_and_prediction(mutation_tool, model, bias_type, mutant)
    evaluation = evaluate_btc(df["label"], df["prediction"],
                              df["mutant"], df["template"], "gender", df["gender"])

    d = d.append(
        {
            "model": model,
            "tool": mutation_tool,
            "template": evaluation['template'],
            "mutant": evaluation['mutant'],
            "btc": evaluation['btc']
        },
        ignore_index=True)

d


,model,tool,template,mutant,btc
0,bert-base-uncased,biasfinder,1672,58388,14824
1,bert-base-cased,biasfinder,1672,58388,14983
2,roberta-base,biasfinder,1672,58388,13439
3,xlnet-base-cased,biasfinder,1672,58388,15551
4,albert-base-v2,biasfinder,1672,58388,9498
5,microsoft/mpnet-base,biasfinder,1672,58388,21406
6,microsoft/deberta-base,biasfinder,1672,58388,15194
7,facebook/muppet-roberta-base,biasfinder,1672,58388,17124
8,google/electra-base-generator,biasfinder,1672,58388,11367
